# Item 45: Consider `@property` Instead of Refactoring Attributes

The `@property` decortor lets us migrate all existing usage of a classto have new behaviors without requiring any of the call sites to be written (which is especially important if there's calling code that we don't have control over). It also provides an important stopgap for improving interfaces overtime.

In [5]:
from datetime import datetime, timedelta

# Leaky bucket quota object
class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f'Bucket(quota={self.quota})'

In [6]:
# Leaky Bucket Algorithm: ensures that whenever the bucket is filled, the amount if quota does not carry
# over from one period to the next
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

In [7]:
# Each time a quota consumer wants to do something, it ust first ensure that it can deduse the amount
# of quota it needs to use
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False # Bucket was filled, but not enough
    bucket.quota -= amount
    return True # Bucket had enough, quota consumed

In [11]:
# Using the above class
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [12]:
# Then, we deduct the quota that we need
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota=1)


In [14]:
# Eventually, we're prevented from making progress because we try to deduct more quota than is available.
# In this case, the bucket's quota level remains unchanged
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Not enough for 3 quota
Bucket(quota=1)


Problem: we never know what quota level the bucket started with. 
Fix: change class to keep track of the `max_quota` issued in the period and the `quota_consumed` in the period:

In [21]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, quota_consumed={self.quota_consumed})')
    
    # To match the previous interface of the original Bucket class, I use a @property method to compute the
    # current level of quota on-the-fly using these new attributes
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    # When the quota attribute is assigned, we take special action to be compatible with the current usage
    # of the class by the fill and deduct functions
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta


In [22]:
# Demo code from above produces the same reuslts
bucket = NewBucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')

print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

print('Still', bucket)

Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now NewBucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
NewBucket(max_quota=100, quota_consumed=99)
Still NewBucket(max_quota=100, quota_consumed=99)
